<a href="https://colab.research.google.com/github/aksh98/Collaborative_Filtering/blob/master/Assignment2_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd drive/My Drive/Colab Notebooks
%cd Assignments_sem7/


/content/drive/My Drive/Colab Notebooks
/content/drive/My Drive/Colab Notebooks/Assignments_sem7


In [0]:
!pip install hpelm

    100% |████████████████████████████████| 51kB 2.0MB/s 
    100% |████████████████████████████████| 163kB 24.9MB/s 
    100% |████████████████████████████████| 3.8MB 971kB/s 
    100% |████████████████████████████████| 163kB 26.3MB/s 


In [0]:

import scipy.io as sio
import numpy as np
from sklearn.metrics import mean_absolute_error as mae
from hpelm import ELM
import pandas as pd
from pandas import DataFrame as df

In [0]:
f =open('ml-100k/u.user','r')
profession_names=[]
profession_id = {}
ages=[]
user_info = []
i = 0
for x in f.readlines():
    if x.split('|')[3] not in profession_names:
        profession_names.append(x.split('|')[3])
        profession_id[x.split('|')[3]]=i
        i+=1
    ages.append(int(x.split('|')[1]))
    user_info.append(x.split('|')[:4])


In [0]:
print(user_info[0])
print(profession_id)
print(min(ages),max(ages))
ages_id = [14,21,29,37,47,55,65,73]
print(ages_id)


['1', '24', 'M', 'technician']
{'technician': 0, 'other': 1, 'writer': 2, 'executive': 3, 'administrator': 4, 'student': 5, 'lawyer': 6, 'educator': 7, 'scientist': 8, 'entertainment': 9, 'programmer': 10, 'librarian': 11, 'homemaker': 12, 'artist': 13, 'engineer': 14, 'marketing': 15, 'none': 16, 'healthcare': 17, 'retired': 18, 'salesman': 19, 'doctor': 20}
7 73
[14, 21, 29, 37, 47, 55, 65, 73]


In [0]:
user_id = 0
user_meta = {}
for i in user_info:
    meta_data = ""
    age = int(i[1])
    gender = i[2]
    profession = i[3]
    temp_age=[0]*len(ages_id)
    if gender == 'M':
        temp_age.append(1)
        temp_age.append(0)
    else:
        temp_age.append(0)
        temp_age.append(1)
    temp_prof = [0]*len(profession_id)
    temp_prof[profession_id[profession]]=1
    for j in range(0,len(profession_id)):
        temp_age.append(temp_prof[j])
    user_meta[user_id]=temp_age
    user_id+=1
    for j in range(0,len(ages_id)):
        if ages_id[j]>age:
            temp_age[j-1]=1
            break
           
user_met = df.from_dict(user_meta)
user_met = user_met.T
print(user_met.shape)
user_met.head()


(943, 31)


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [0]:
## MOVIES ------------------

f=open('ml-100k/u.item','r',encoding = "ISO-8859-1")
item_meta = {}

for x in f.readlines():
    idx = int(x.split('|')[0])
    temp = []
    for y in x.split('|')[5:]:
        if y.endswith("\n"): 
            y = int(y[:-1])
        temp.append(int(y))
    item_meta[idx]= temp
# print(len(item_meta))
item_met = df.from_dict(item_meta)
item_met = item_met.T
print(item_met.shape)
item_met.head()
# item_meta.shape
# print(train_set.shape,train_ids.shape)

(1682, 19)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


# ELM


Parameters Used - 
- 25 layers with an increment of 10 neurons each, starting from 10 upto 250.
- RBF_l2 gives the best accuracy amongst the ones I tried.

In [0]:
##   RATINGS ------------------

f = open('ml-100k/u.data','r')
# ratings = {}
# ratings = df()
X = []
Y = []
count = 0
for x in f.readlines():
    idx = x.split()
    temp = []
    temp_rat = [0]*5
    temp_rat[int(idx[2])-1] = 1
    temp = sum([user_meta[int(idx[0])-1],item_meta[int(idx[1])]], [])  
    X.append(temp)
    Y.append(temp_rat)
print(len(X),len(X[0]),len(Y),len(Y[0]))
print(X[0])
print(Y[0])

100000 50 100000 5
[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0]



Accuracy score - 200 neurons :
- Fold 1
  - Accuracy Score linf:  0.345
  - Accuracy Score l2:    0.345
  - Accuracy Score sigm:  0.344

- Fold 2 
  - Accuracy Score linf:  0.352
  - Accuracy Score l2:    0.3531
  - Accuracy Score sigm:  0.352
  
- Fold 3
  - Accuracy Score linf:  0.3568
  - Accuracy Score l2:    0.3574
  - Accuracy Score sigm:  0.356

- Fold 4
  - Accuracy Score linf:  0.356
  - Accuracy Score l2:  0.357
  - Accuracy Score sigm:  0.357
  
-  Fold 5
  - Accuracy Score linf:  0.3557
  - Accuracy Score l2:  0.356
  - Accuracy Score sigm:  0.356
  
### MAE for each - 
  - MAE linf:  0.218
  - MAE l2:  0.2175
  - MAE sigm:  0.217

### Final Accuracy scores 
- **For rbf_linf:  0.353**
- **For rbf_l2:  0.35388**
- **For Sigm:  0.35307 **

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

X_train = []
y_train = []
X_test = []
y_test = []

linf = []
l2 = []
sigm = []

k_folds = KFold(n_splits = 5,shuffle=False)
for train_index, test_index in k_folds.split(X):
    print("TRAIN:", train_index[0], "          TEST:", test_index[0])
    for i in train_index:
        X_train.append(X[i])
        y_train.append(Y[i])
    for i in test_index:
        X_test.append(X[i])
        y_test.append(Y[i])
    
    #############################
#     20,1500,100
#     5,30,5
# .  train ratings, train features
#     10,200,30
    accuracy = []
    error = []

    for x in range(10,250,10):
        elm_algo = ELM(len(X_train[0]),len(y_train[0]))
        # TO TRY : #rbf_l2 #sigm #tanh
        elm_algo.add_neurons(x, "rbf_linf") 
        elm_algo.train(np.asarray(X_train),np.asarray(y_train))
        pred = elm_algo.predict(np.asarray(X_test))
        Y_predicted = np.argmax(pred, axis=1)
        Y_true = np.argmax(y_test, axis=1)
        
        err = mae(Y_true,Y_predicted)/4
        acc = accuracy_score(Y_true,Y_predicted)
        accuracy.append(acc)
        error.append(err)
    linf.append(sum(accuracy)/len(accuracy))
    print("Accuracy Score linf: ",sum(accuracy)/len(accuracy))
    print("MAE linf: ", sum(error)/len(error))

    accuracy = []
    error = []
    
    for x in range(10,250,10):
        elm_algo = ELM(len(X_train[0]),len(y_train[0]))
        # TO TRY : #rbf_l2 #sigm #tanh
        elm_algo.add_neurons(x, "rbf_l2") 
        elm_algo.train(np.asarray(X_train),np.asarray(y_train))
        pred = elm_algo.predict(np.asarray(X_test))
        Y_predicted = np.argmax(pred, axis=1)
        Y_true = np.argmax(y_test, axis=1)
        
        err = mae(Y_true,Y_predicted)/4
        acc = accuracy_score(Y_true,Y_predicted)
        accuracy.append(acc)
        error.append(err)
    print("Accuracy Score l2: ",sum(accuracy)/len(accuracy))
    print("MAE l2: ", sum(error)/len(error))
    l2.append(sum(accuracy)/len(accuracy))
    
    accuracy = []
    error = []

    for x in range(10,250,10):
        elm_algo = ELM(len(X_train[0]),len(y_train[0]))
        # TO TRY : #rbf_l2 #sigm #tanh
        elm_algo.add_neurons(x, "sigm") 
        elm_algo.train(np.asarray(X_train),np.asarray(y_train))
        pred = elm_algo.predict(np.asarray(X_test))
        Y_predicted = np.argmax(pred, axis=1)
        Y_true = np.argmax(y_test, axis=1)
        
        err = mae(Y_true,Y_predicted)/4
        acc = accuracy_score(Y_true,Y_predicted)
        accuracy.append(acc)
        error.append(err)
    sigm.append(sum(accuracy)/len(accuracy))
    print("Accuracy Score sigm: ",sum(accuracy)/len(accuracy))
    print("MAE sigm: ", sum(error)/len(error))
    
print()
print("Final Accuracy scores ")
print("For rbf_linf: ",sum(linf)/len(linf))
print("For rbf_l2: ",sum(l2)/len(l2))
print("For Sigm: ",sum(sigm)/len(sigm))


TRAIN: 20000           TEST: 0
Covariance matrix is not full rank; solving with SVD (slow)
This happened because you have duplicated or too many neurons


/usr/local/lib/python3.6/dist-packages/hpelm/nnets/slfn_python.py:65: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  B = np.linalg.lstsq(HH, HT)[0]


Covariance matrix is not full rank; solving with SVD (slow)
This happened because you have duplicated or too many neurons
Covariance matrix is not full rank; solving with SVD (slow)
This happened because you have duplicated or too many neurons
Covariance matrix is not full rank; solving with SVD (slow)
This happened because you have duplicated or too many neurons
Covariance matrix is not full rank; solving with SVD (slow)
This happened because you have duplicated or too many neurons
Covariance matrix is not full rank; solving with SVD (slow)
This happened because you have duplicated or too many neurons
Covariance matrix is not full rank; solving with SVD (slow)
This happened because you have duplicated or too many neurons
Covariance matrix is not full rank; solving with SVD (slow)
This happened because you have duplicated or too many neurons
Accuracy Score linf:  0.3446375000000001
MAE linf:  0.22365989583333334
Accuracy Score l2:  0.34558958333333334
MAE l2:  0.2231421875
Accuracy Sco

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

X_train = []
y_train = []
X_test = []
y_test = []

linf = []
l2 = []
sigm = []

k_folds = KFold(n_splits = 5,shuffle=False)
for train_index, test_index in k_folds.split(X):
    print("TRAIN:", train_index[0], "          TEST:", test_index[0])
    for i in train_index:
        X_train.append(X[i])
        y_train.append(Y[i])
    for i in test_index:
        X_test.append(X[i])
        y_test.append(Y[i])
    
    #############################
#     20,1500,100
#     5,30,5
# .  train ratings, train features
#     10,200,30
    accuracy = []
    error = []

    for x in range(10,250,10):
        elm_algo = ELM(len(X_train[0]),len(y_train[0]))
        # TO TRY : #rbf_l2 #sigm #tanh
        elm_algo.add_neurons(x, "rbf_linf") 
        elm_algo.train(np.asarray(X_train),np.asarray(y_train))
        pred = elm_algo.predict(np.asarray(X_test))
        Y_predicted = np.argmax(pred, axis=1)
        Y_true = np.argmax(y_test, axis=1)
        
        err = mae(Y_true,Y_predicted)/4
        acc = accuracy_score(Y_true,Y_predicted)
        accuracy.append(acc)
        error.append(err)
    linf.append(sum(accuracy)/len(accuracy))
    print("Accuracy Score linf: ",sum(accuracy)/len(accuracy))
    print("MAE linf: ", sum(error)/len(error))

    accuracy = []
    error = []
    
    for x in range(10,250,10):
        elm_algo = ELM(len(X_train[0]),len(y_train[0]))
        # TO TRY : #rbf_l2 #sigm #tanh
        elm_algo.add_neurons(x, "rbf_l2") 
        
        elm_algo.train(np.asarray(X_train),np.asarray(y_train))
        pred = elm_algo.predict(np.asarray(X_test))
        Y_predicted = np.argmax(pred, axis=1)
        Y_true = np.argmax(y_test, axis=1)
        
        err = mae(Y_true,Y_predicted)/4
        acc = accuracy_score(Y_true,Y_predicted)
        accuracy.append(acc)
        error.append(err)
    print("Accuracy Score l2: ",sum(accuracy)/len(accuracy))
    print("MAE l2: ", sum(error)/len(error))
    l2.append(sum(accuracy)/len(accuracy))
    
    accuracy = []
    error = []

    for x in range(10,250,10):
        elm_algo = ELM(len(X_train[0]),len(y_train[0]))
        # TO TRY : #rbf_l2 #sigm #tanh
        elm_algo.add_neurons(x, "sigm") 
        elm_algo.train(np.asarray(X_train),np.asarray(y_train))
        pred = elm_algo.predict(np.asarray(X_test))
        Y_predicted = np.argmax(pred, axis=1)
        Y_true = np.argmax(y_test, axis=1)
        
        err = mae(Y_true,Y_predicted)/4
        acc = accuracy_score(Y_true,Y_predicted)
        accuracy.append(acc)
        error.append(err)
    sigm.append(sum(accuracy)/len(accuracy))
    print("Accuracy Score sigm: ",sum(accuracy)/len(accuracy))
    print("MAE sigm: ", sum(error)/len(error))
    
print()
print("Final Accuracy scores ")
print("For rbf_linf: ",sum(linf)/len(linf))
print("For rbf_l2: ",sum(l2)/len(l2))
print("For Sigm: ",sum(sigm)/len(sigm))


# SVM 


### Parameters Used
- Gamma = 'auto'
- C=0.1
- Kernel = 'rbf'


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

##   RATINGS ------------------

f = open('ml-100k/u.data','r')
# ratings = {}
# ratings = df()
X = []
Y = []
count = 0
for x in f.readlines():
    idx = x.split()
    temp = []
    temp_rat = idx[2]
#     print(temp_rat)
#     temp_rat[int(idx[2])-1] = 1
#     print(temp_rat)
    temp = sum([user_meta[int(idx[0])-1],item_meta[int(idx[1])]], [])  
    X.append(temp)
    Y.append(temp_rat)
print(len(X),len(X[0]),len(Y),len(Y[0]))
print(X[0])
print(Y[0])

100000 50 100000 1
[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3


### Fold 1 ->
- Accuracy - 0.3389

### Fold 2 ->
-

### Fold 3 ->
-

### Fold 4 ->
-

### Fold 5 ->
- 

In [0]:
from sklearn.svm import SVC

X_train = []
y_train = []
X_test = []
y_test = []

rbf = []

k_folds = KFold(n_splits = 5,shuffle=False)
for train_index, test_index in k_folds.split(X):
    print("TRAIN:", train_index[0], "     TEST:", test_index[0])
    for i in train_index:
        X_train.append(X[i])
        y_train.append(Y[i])
    for i in test_index:
        X_test.append(X[i])
        y_test.append(Y[i])
#     print("-",len(X_test),len(y_test),len(y_test[0]))
    #############################
#     5,30,5
# .  train ratings, train features
#     10,200,30
    accuracy = []
    error = []
  
    svc = SVC(C = 0.1,gamma='auto',kernel='rbf') #gamma = 'scale'
    svc.fit(X_train,y_train)
    pred = svc.predict(X_test)
    acc = accuracy_score(y_test,pred)  
    print("Accuracy: ",acc)
    rbf.append(acc)
    
print("AVG Accuracy -> ",sum(rbf)/len(rbf))

TRAIN: 20000      TEST: 0
Accuracy:  0.3389
TRAIN: 0      TEST: 20000
